In [2]:
import dill as pickle
from collections import Counter, defaultdict
import csv
import re
import numpy as np
import os
from evlex_shared_scripts import *

In [3]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = TFBertModel.from_pretrained('bert-base-cased')

2025-04-24 09:18:35.672743: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-24 09:18:35.766170: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745500715.802794 1019611 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745500715.813369 1019611 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-24 09:18:35.893728: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [4]:
def get_indices(fn):
    indices = []
    i = 0
    bitext = [' '.join([w.split('/')[0] for w in l.strip('\n').split(' ||| ')[1].split()]) for l in open(fn).readlines()]
    print(fn, bitext[0])
    for ix,ft in enumerate(bitext):
        input_id = tf.constant(tokenizer.encode(ft))[None, :]
        if i == 0 :
            print(ix, ft, '\n', [tokenizer.decode([iii]) for iii in input_id[:,(0 if ix == 0 else 1):].numpy()[0]],'\n',
            tokenizer.decode([iii for iii in input_id[:,(0 if ix == 0 else 1):].numpy()[0]]))
        if ix > 0: 
            input_id = input_id[:,1:]
        #
        if ix == 0: 
            input_ids = input_id
        else: 
            input_ids = tf.concat((input_ids, input_id), 1)
        indices.extend([ix] * input_id.shape[1])
        i += 1
    return indices, input_ids

In [5]:
def get_slices(indices, padding = 100, max_tok = 512):
    start = 0
    end = len(indices[0])
    s_outer, e_outer = 0, min(max_tok, end)
    s_inner, e_inner = s_outer, e_outer - (padding if e_outer != end else 0)
    slices = []
    slices.append(((s_outer, e_outer),(s_inner - s_outer, (e_outer-s_outer) - (e_outer-e_inner))))
    while s_outer != end and e_inner != end:
        s_outer = min(end, s_outer + (max_tok - 2 * padding))
        e_outer = min(end, e_outer + (max_tok - 2 * padding))
        s_inner = s_outer + padding
        e_inner = e_outer - (padding if e_outer != end else 0)
        if e_inner == end:
            s_outer = max(0, e_outer-max_tok)
        
        slices.append(((s_outer, e_outer),(s_inner - s_outer, (e_outer-s_outer) - (e_outer-e_inner))))
    #print((s_outer, e_outer),(s_inner,e_inner))
    return slices

In [9]:
from datetime import datetime
dataset = 'doreco'
if dataset == 'opus': languages = ['el', 'fi', 'tr', 'es'] 
else: languages = [l.strip('\n') for l in open('doreco_doculects.txt')]
outdir = dn = './generated/%s_bert/' % dataset

indices = defaultdict(lambda : [])
input_ids = {}
chunk_size = 100000
for language in languages:
    #
    chunk_ix = 0
    bert = None
    #
    fn = './generated/%s_bitexts/%s.spc' % (dataset, language)
    indices[language], input_ids[language] = get_indices(fn)
    slices = get_slices(input_ids[language])
    for (so,eo),(si,ei) in slices:
        input_id = input_ids[language][:,so:eo]
        outputs = model(input_id)
        if bert == None:
            bert = outputs[0][:,si:ei,:]
        else:
            bert = tf.concat((bert, outputs[0][:,si:ei,:]), 1)
        if bert.shape[1] > chunk_size:
            print('\t',chunk_ix,input_ids[language].shape,bert.shape, datetime.now())
            np.save('%s/%s_%d.npy' % (outdir,language[:8],chunk_ix), bert.numpy())
            chunk_ix += 1
            bert = None
    np.save('%s/%s_%d.npy' % (outdir,language[:8],chunk_ix), bert.numpy())
    print('\t',chunk_ix,input_ids[language].shape,bert.shape, datetime.now())
pickle.dump(indices, open('%s/indices.p' % outdir, 'wb'))
pickle.dump(input_ids, open('%s/input_ids.p' % outdir, 'wb'))

./generated/doreco_bitexts/apah1238.spc mm
0 mm 
 ['[CLS]', 'mm', '[SEP]'] 
 [CLS] mm [SEP]
	 0 (1, 19243) (1, 19243, 768) 2025-04-24 19:34:42.309092
./generated/doreco_bitexts/anal1239.spc Anal proverbs
0 Anal proverbs 
 ['[CLS]', 'Ana', '##l', 'prove', '##rb', '##s', '[SEP]'] 
 [CLS] Anal proverbs [SEP]
	 0 (1, 22254) (1, 22254, 768) 2025-04-24 19:35:10.119986
./generated/doreco_bitexts/arap1274.spc Trickster was walking along
0 Trickster was walking along 
 ['[CLS]', 'Trick', '##ster', 'was', 'walking', 'along', '[SEP]'] 
 [CLS] Trickster was walking along [SEP]
	 0 (1, 36919) (1, 36919, 768) 2025-04-24 19:35:59.163286
./generated/doreco_bitexts/beja1238.spc A man :
0 A man : 
 ['[CLS]', 'A', 'man', ':', '[SEP]'] 
 [CLS] A man : [SEP]
	 0 (1, 41949) (1, 41949, 768) 2025-04-24 19:36:55.449384
./generated/doreco_bitexts/cabe1245.spc Long ago they lived under the trees
0 Long ago they lived under the trees 
 ['[CLS]', 'Long', 'ago', 'they', 'lived', 'under', 'the', 'trees', '[SEP]'] 
 